In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

c:\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = pd.read_csv("D:/data/CRF Final Data/oscar/main oscar.csv")
data = data.set_index("ID")
data.head()

,Unnamed: 0,target,peakflow,वय_275546,लिंग: _275544,शिक्षण_275545,आपल्या घरातील कुटुंबातील सदस्यांची संख्या? _275550,घराविषयीची माहिती 0 तुमचे घर कोणत्या प्रकारचे आहे ?_275549,तुमच्या घरात किती खोल्या आहेत?_275551,तुमच्या घराला एकूण किती खिडक्या आहेत ?_275552,...,"जर हो, किती वर्ष पासून? _275700",हृदय रोग_275701,"जर हो, किती वर्ष पासून? _275702",थाइरोइड विकार_275703,"जर हो, किती वर्ष पासून? _275704",चिंता आणि निराशा_275705,"जर हो, किती वर्ष पासून? _275706",हाड ठिसूळ होणे_275707,"जर हो, किती वर्ष पासून? _275708",धन्यवाद सबमिट करा_275709
ID,,,,,,,,,,,,,,,,,,,,,
A1,0,1,210,37,2,1,4,2,2,3,...,0,0,0,0,0,0,0,0,0,0
A2,1,0,550,33,2,3,4,2,2,3,...,0,0,0,0,0,0,0,0,0,0
A3,2,0,290,19,1,4,4,2,2,3,...,0,0,0,0,0,0,0,0,0,0
A4,3,0,340,44,2,3,4,2,3,3,...,0,0,0,0,0,0,0,0,0,0
A5,4,0,420,38,2,3,5,2,3,3,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = data.drop('target', axis = 1)
y = data['target']

X.shape, y.shape

((798, 182), (798,))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)


# Remove Constant, Quasi Constant and Duplicate Features

In [5]:
#remove constant and quasi constant features
constant_filter = VarianceThreshold(threshold=0.01)
constant_filter.fit(X_train)
X_train_filter = constant_filter.transform(X_train)
X_test_filter = constant_filter.transform(X_test)

In [6]:
X_train_filter

array([[319, 400,  34, ...,   0,   0,   0],
       [570, 300,  45, ...,   0,   0,   0],
       [638, 260,  37, ...,   0,   0,   0],
       ...,
       [571, 270,  43, ...,   0,   1,   2],
       [786, 250,  77, ...,   5,   1,   4],
       [686, 460,  62, ...,   0,   0,   0]], dtype=int64)

In [7]:
X_train_filter.shape, X_test_filter.shape

((638, 146), (160, 146))

In [8]:

#remove duplicate features
X_train_T = X_train_filter.T
X_test_T = X_test_filter.T

In [9]:
X_train_T

array([[319, 570, 638, ..., 571, 786, 686],
       [400, 300, 260, ..., 270, 250, 460],
       [ 34,  45,  37, ...,  43,  77,  62],
       ...,
       [  0,   0,   0, ...,   0,   5,   0],
       [  0,   0,   0, ...,   1,   1,   0],
       [  0,   0,   0, ...,   2,   4,   0]], dtype=int64)

In [10]:
X_train_T = pd.DataFrame(X_train_T)
X_test_T = pd.DataFrame(X_test_T)

In [11]:
X_train_T.duplicated().sum()

0

In [12]:

duplicated_features = X_train_T.duplicated()

In [13]:
features_to_keep = [not index for index in duplicated_features]

X_train_unique = X_train_T[features_to_keep].T
X_test_unique = X_test_T[features_to_keep].T

In [14]:
scaler = StandardScaler().fit(X_train_unique)
X_train_unique = scaler.transform(X_train_unique)
X_test_unique = scaler.transform(X_test_unique)

In [15]:
X_train_unique = pd.DataFrame(X_train_unique)
X_test_unique = pd.DataFrame(X_test_unique)

In [16]:

X_train_unique.shape, X_test_unique.shape

((638, 146), (160, 146))

# Removal of correlated Feature

In [17]:
corrmat = X_train_unique.corr()

In [18]:
def get_correlation(data, threshold):
    corr_col = set()
    corrmat = data.corr()
    for i in range(len(corrmat.columns)):
        for j in range(i):
            if abs(corrmat.iloc[i, j]) > threshold:
                colname = corrmat.columns[i]
                corr_col.add(colname)
    return corr_col

corr_features = get_correlation(X_train_unique, 0.70)
print('correlated features: ', len(set(corr_features)) )

correlated features:  42


In [19]:
X_train_uncorr = X_train_unique.drop(labels=corr_features, axis = 1)
X_test_uncorr = X_test_unique.drop(labels = corr_features, axis = 1)

In [20]:
X_train_uncorr.shape, X_test_uncorr.shape

((638, 104), (160, 104))

# Feature Dimention Reduction by LDA or Is it a Classifier

In [21]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [22]:
lda = LDA(n_components=1)
X_train_lda = lda.fit_transform(X_train_uncorr, y_train)
X_test_lda = lda.transform(X_test_uncorr)

In [23]:
X_train_lda.shape, X_test_lda.shape

((638, 1), (160, 1))

In [48]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def run_randomForest(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy on test set: ')
    print("Accuracy: ", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    
    

In [49]:

%%time
run_randomForest(X_train_lda, X_test_lda, y_train, y_test)

Accuracy on test set: 
Accuracy:  0.69375
              precision    recall  f1-score   support

           0       0.84      0.79      0.81       135
           1       0.12      0.16      0.14        25

    accuracy                           0.69       160
   macro avg       0.48      0.48      0.48       160
weighted avg       0.72      0.69      0.71       160

[[107  28]
 [ 21   4]]
Wall time: 578 ms


In [50]:
%%time
run_randomForest(X_train_uncorr, X_test_uncorr, y_train, y_test)

Accuracy on test set: 
Accuracy:  0.84375
              precision    recall  f1-score   support

           0       0.84      1.00      0.92       135
           1       0.00      0.00      0.00        25

    accuracy                           0.84       160
   macro avg       0.42      0.50      0.46       160
weighted avg       0.71      0.84      0.77       160

[[135   0]
 [ 25   0]]
Wall time: 637 ms


c:\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
%%time
run_randomForest(X_train, X_test, y_train, y_test)

Accuracy on test set: 
Accuracy:  0.84375
              precision    recall  f1-score   support

           0       0.84      1.00      0.92       135
           1       0.00      0.00      0.00        25

    accuracy                           0.84       160
   macro avg       0.42      0.50      0.46       160
weighted avg       0.71      0.84      0.77       160

[[135   0]
 [ 25   0]]
Wall time: 647 ms


c:\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Feature Reduction by PCA?

In [52]:

from sklearn.decomposition import PCA

In [53]:
pca = PCA(n_components=10, random_state=42)
pca.fit(X_train_uncorr)

PCA(copy=True, iterated_power='auto', n_components=10, random_state=42,
    svd_solver='auto', tol=0.0, whiten=False)

In [54]:
X_train_pca = pca.transform(X_train_uncorr)
X_test_pca = pca.transform(X_test_uncorr)
X_train_pca.shape, X_test_pca.shape

((638, 10), (160, 10))

In [55]:
%%time
run_randomForest(X_train_pca, X_test_pca, y_train, y_test)

Accuracy on test set: 
Accuracy:  0.84375
              precision    recall  f1-score   support

           0       0.84      1.00      0.92       135
           1       0.00      0.00      0.00        25

    accuracy                           0.84       160
   macro avg       0.42      0.50      0.46       160
weighted avg       0.71      0.84      0.77       160

[[135   0]
 [ 25   0]]
Wall time: 565 ms


c:\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:

%%time
run_randomForest(X_train, X_test, y_train, y_test)

Accuracy on test set: 
Accuracy:  0.84375
              precision    recall  f1-score   support

           0       0.84      1.00      0.92       135
           1       0.00      0.00      0.00        25

    accuracy                           0.84       160
   macro avg       0.42      0.50      0.46       160
weighted avg       0.71      0.84      0.77       160

[[135   0]
 [ 25   0]]
Wall time: 689 ms


c:\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
X_train_uncorr.shape

(638, 104)

In [58]:
for component in range(1,30):
    pca = PCA(n_components=component, random_state=42)
    pca.fit(X_train_uncorr)
    X_train_pca = pca.transform(X_train_uncorr)
    X_test_pca = pca.transform(X_test_uncorr)
    print('Selected Components: ', component)
    run_randomForest(X_train_pca, X_test_pca, y_train, y_test)
    print()

Selected Components:  1
Accuracy on test set: 
Accuracy:  0.74375
              precision    recall  f1-score   support

           0       0.85      0.84      0.85       135
           1       0.19      0.20      0.20        25

    accuracy                           0.74       160
   macro avg       0.52      0.52      0.52       160
weighted avg       0.75      0.74      0.75       160

[[114  21]
 [ 20   5]]

Selected Components:  2
Accuracy on test set: 
Accuracy:  0.80625
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       135
           1       0.20      0.08      0.11        25

    accuracy                           0.81       160
   macro avg       0.52      0.51      0.50       160
weighted avg       0.75      0.81      0.77       160

[[127   8]
 [ 23   2]]

Selected Components:  3
Accuracy on test set: 
Accuracy:  0.83125
              precision    recall  f1-score   support

           0       0.85      0.98      0.91  

c:\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy on test set: 
Accuracy:  0.8375
              precision    recall  f1-score   support

           0       0.84      0.99      0.91       135
           1       0.00      0.00      0.00        25

    accuracy                           0.84       160
   macro avg       0.42      0.50      0.46       160
weighted avg       0.71      0.84      0.77       160

[[134   1]
 [ 25   0]]

Selected Components:  10
Accuracy on test set: 
Accuracy:  0.84375
              precision    recall  f1-score   support

           0       0.84      1.00      0.92       135
           1       0.00      0.00      0.00        25

    accuracy                           0.84       160
   macro avg       0.42      0.50      0.46       160
weighted avg       0.71      0.84      0.77       160

[[135   0]
 [ 25   0]]

Selected Components:  11


c:\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy on test set: 
Accuracy:  0.85625
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       135
           1       1.00      0.08      0.15        25

    accuracy                           0.86       160
   macro avg       0.93      0.54      0.53       160
weighted avg       0.88      0.86      0.80       160

[[135   0]
 [ 23   2]]

Selected Components:  12
Accuracy on test set: 
Accuracy:  0.84375
              precision    recall  f1-score   support

           0       0.84      1.00      0.92       135
           1       0.00      0.00      0.00        25

    accuracy                           0.84       160
   macro avg       0.42      0.50      0.46       160
weighted avg       0.71      0.84      0.77       160

[[135   0]
 [ 25   0]]

Selected Components:  13


...

Accuracy on test set: 
Accuracy:  0.85
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       135
           1       1.00      0.04      0.08        25

    accuracy                           0.85       160
   macro avg       0.92      0.52      0.50       160
weighted avg       0.87      0.85      0.79       160

[[135   0]
 [ 24   1]]

Selected Components:  17
Accuracy on test set: 
Accuracy:  0.85
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       135
           1       1.00      0.04      0.08        25

    accuracy                           0.85       160
   macro avg       0.92      0.52      0.50       160
weighted avg       0.87      0.85      0.79       160

[[135   0]
 [ 24   1]]

Selected Components:  18
Accuracy on test set: 
Accuracy:  0.84375
              precision    recall  f1-score   support

           0       0.85      0.99      0.91       135
           1       

c:\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy on test set: 
Accuracy:  0.85
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       135
           1       1.00      0.04      0.08        25

    accuracy                           0.85       160
   macro avg       0.92      0.52      0.50       160
weighted avg       0.87      0.85      0.79       160

[[135   0]
 [ 24   1]]

Selected Components:  21
Accuracy on test set: 
Accuracy:  0.85
              precision    recall  f1-score   support

           0       0.85      0.99      0.92       135
           1       0.67      0.08      0.14        25

    accuracy                           0.85       160
   macro avg       0.76      0.54      0.53       160
weighted avg       0.82      0.85      0.80       160

[[134   1]
 [ 23   2]]

Selected Components:  22
Accuracy on test set: 
Accuracy:  0.85
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       135
           1       1.0

c:\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy on test set: 
Accuracy:  0.85
              precision    recall  f1-score   support

           0       0.85      0.99      0.92       135
           1       0.67      0.08      0.14        25

    accuracy                           0.85       160
   macro avg       0.76      0.54      0.53       160
weighted avg       0.82      0.85      0.80       160

[[134   1]
 [ 23   2]]

Selected Components:  27
Accuracy on test set: 
Accuracy:  0.85
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       135
           1       1.00      0.04      0.08        25

    accuracy                           0.85       160
   macro avg       0.92      0.52      0.50       160
weighted avg       0.87      0.85      0.79       160

[[135   0]
 [ 24   1]]

Selected Components:  28
Accuracy on test set: 
Accuracy:  0.8625
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       135
           1       1

# RIDGE LASSO

In [2]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

c:\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
data = pd.read_csv("D:/data/CRF Final Data/oscar/main oscar.csv")
data = data.set_index("ID")
data.head()

,Unnamed: 0,target,peakflow,वय_275546,लिंग: _275544,शिक्षण_275545,आपल्या घरातील कुटुंबातील सदस्यांची संख्या? _275550,घराविषयीची माहिती 0 तुमचे घर कोणत्या प्रकारचे आहे ?_275549,तुमच्या घरात किती खोल्या आहेत?_275551,तुमच्या घराला एकूण किती खिडक्या आहेत ?_275552,...,"जर हो, किती वर्ष पासून? _275700",हृदय रोग_275701,"जर हो, किती वर्ष पासून? _275702",थाइरोइड विकार_275703,"जर हो, किती वर्ष पासून? _275704",चिंता आणि निराशा_275705,"जर हो, किती वर्ष पासून? _275706",हाड ठिसूळ होणे_275707,"जर हो, किती वर्ष पासून? _275708",धन्यवाद सबमिट करा_275709
ID,,,,,,,,,,,,,,,,,,,,,
A1,0,1,210,37,2,1,4,2,2,3,...,0,0,0,0,0,0,0,0,0,0
A2,1,0,550,33,2,3,4,2,2,3,...,0,0,0,0,0,0,0,0,0,0
A3,2,0,290,19,1,4,4,2,2,3,...,0,0,0,0,0,0,0,0,0,0
A4,3,0,340,44,2,3,4,2,3,3,...,0,0,0,0,0,0,0,0,0,0
A5,4,0,420,38,2,3,5,2,3,3,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = data.drop('target', axis = 1)
y = data['target']

X.shape, y.shape

((798, 182), (798,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [6]:
sel = SelectFromModel(LogisticRegression(C = 0.05, penalty = 'l1', solver = 'liblinear'))
sel.fit(X_train, y_train)

SelectFromModel(estimator=LogisticRegression(C=0.05, class_weight=None,
                                             dual=False, fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [7]:
sel.get_support()

array([ True,  True,  True, False, False,  True, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False,  True, False, False, False, False,  True,
       False, False,  True, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False, False,  True, False, False,  True, False, False,  True,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,

In [8]:
features = X_train.columns[sel.get_support()]
features

Index(['Unnamed: 0', 'peakflow', 'वय_275546',
       'आपल्या घरातील कुटुंबातील सदस्यांची संख्या? _275550',
       'तुमच्या घरात किती खोल्या आहेत?_275551', 'बिडी संख्या / दर महिना',
       'जर हो, किती वर्षे पासून धूम्रपान करत आहे?_275573',
       'जर होय, तर तुमच्या घरात, घराच्या आतमध्ये धूम्रपान किती वर्षांपासून करतात ?_275577',
       'वापरलेले वर्षे', 'वापरलेले तास.2',
       'तुम्ही स्वतः किती वर्षापसून स्वयंपाक करता / करत होता ?_275610',
       'व्यवसाय', 'कालावधी.2', 'किती वर्षे', 'किती वर्षे.2', 'किती वर्षे.3',
       'किती वर्षे.5', 'किती वर्षे.6', 'किती वर्षे.7',
       'जर होय, तर 'दमा' हा आजार तुम्हाला किती वर्षापासून आहे ?_275651',
       'तुमचा खोकला दिवसभरात कोणत्यावेळी सर्वात जास्त प्रमाणात होता / आहे ?_276711_दुपारी',
       'तुम्हाला सपाट जमिनीवर सुमारे किती मीटर चालल्यानंतर दम लागतो ?_275674',
       'जर होय, तर कोणत्या ऋतूत तुमचा दम वाढतो ?_275683_पावसाळी',
       'तुम्हाला किती वर्षापासून दम लागतो ?_275680',
       'तुमच्या कुटुंबात कधी कोणाला नमुद आजार झाला होता / 

In [9]:

X_train_l1 = sel.transform(X_train)
X_test_l1 = sel.transform(X_test)

In [10]:

X_train_l1.shape, X_test_l1.shape

((534, 26), (264, 26))

In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def run_randomForest(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs = -1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
   
    
    print('Accuracy on test set: ')
    print("Accuracy: ", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    
    

In [12]:
%%time
run_randomForest(X_train_l1, X_test_l1, y_train, y_test)

Accuracy on test set: 
Accuracy:  0.8446969696969697
              precision    recall  f1-score   support

           0       0.86      0.98      0.91       224
           1       0.43      0.07      0.13        40

    accuracy                           0.84       264
   macro avg       0.64      0.53      0.52       264
weighted avg       0.79      0.84      0.80       264

[[220   4]
 [ 37   3]]
Wall time: 835 ms


In [13]:
%%time
run_randomForest(X_train, X_test, y_train, y_test)

Accuracy on test set: 
Accuracy:  0.8560606060606061
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       224
           1       0.75      0.07      0.14        40

    accuracy                           0.86       264
   macro avg       0.80      0.54      0.53       264
weighted avg       0.84      0.86      0.80       264

[[223   1]
 [ 37   3]]
Wall time: 610 ms


# Ridge Regression

In [15]:
from sklearn.linear_model import RidgeClassifierCV

In [16]:
rr = RidgeClassifierCV(alphas=[10, 20, 50, 100, 200, 300], cv = 10 )
rr.fit(X_train, y_train)

RidgeClassifierCV(alphas=array([ 10,  20,  50, 100, 200, 300]),
                  class_weight=None, cv=10, fit_intercept=True, normalize=False,
                  scoring=None, store_cv_values=False)

In [17]:
rr.score(X_test, y_test)

0.8409090909090909

In [18]:

rr.coef_

array([[-1.89039320e-04, -8.58307210e-05,  4.74591322e-03,
        -4.35851589e-03,  1.66619748e-02, -4.05704592e-03,
        -1.46781738e-02, -3.00751452e-02,  1.60063565e-02,
        -1.62577749e-02,  2.46589524e-02, -8.28171074e-03,
         3.25767252e-02, -7.15768425e-04,  3.48836391e-03,
         0.00000000e+00,  9.26357610e-04, -2.90109902e-03,
         3.19459022e-03, -4.43898932e-03, -1.41606658e-02,
         0.00000000e+00, -8.24374593e-04,  3.20111525e-03,
        -3.44088498e-03, -1.17851975e-04,  3.65462939e-03,
         7.64490709e-03,  5.09660473e-02,  9.81248153e-04,
         1.44535256e-02, -1.06106523e-02, -1.87797150e-03,
        -6.10518150e-03, -1.83024503e-04, -2.09210075e-02,
         0.00000000e+00,  1.73597818e-03, -1.39017935e-03,
        -1.08048102e-02,  0.00000000e+00, -1.87899151e-02,
         3.00664579e-03, -5.00142717e-04,  0.00000000e+00,
        -1.68001988e-02, -2.20943242e-04,  4.26449862e-02,
         0.00000000e+00, -1.55976723e-02, -8.07154190e-0

In [19]:
rr.alpha_

300

In [20]:
rr.alphas

array([ 10,  20,  50, 100, 200, 300])